In [1]:
suppressPackageStartupMessages({ 
    library(data.table) 
    library(dplyr) 
    library(ggplot2) 
    library(SingleCellExperiment)
    library(dplyr)
    library(celldex)
    library(SingleR)
    library(RColorBrewer)
    library(scater) 
    library(StabMap) 
    library(scran) 
    library(harmony) 
    library(patchwork)
    library(Seurat)
    library(plotly)
    library(pheatmap)
    library(batchelor)
    library(ggpubr)
    library(cowplot)
    library(viridis)
    library(Matrix)
        library(patchwork)
    library(Cairo)
    library(grid)
        library(png)
})

## Data Locations and Loading in Datasets

In [2]:
##### Load in Seurat Objects for post QC analyses #####

io = list()
io$main = "/rds/project/rds-SDzz0CATGms/users/ltgh2" # this is the main directory

# Set the working directory
setwd(io$main)

io$meta = file.path(io$main, "projects/03_seqFISH_FINAL/code/FINAL_METADATA/outputs/total_meta_updated.Rds")

# Load in the metadata
meta = readRDS(io$meta)

In [3]:
# load in the cell type annotations that were given to all seqFISH cells in Harland et al.

# for the seqFISH cells 
io$seqFISH_celltype_seqFISH = file.path(io$main, "projects/03_seqFISH_FINAL/code/stabmap_label_transfer/output/StabMAP_all_stages_post_QC_reweighted_MNNcorrected_original_counts_without_neigh_postQC_annotations_to_scRNA_atlas.Rds")
seqFISH_celltype_for_scRNA = readRDS(io$seqFISH_celltype_seqFISH)

# for the scRNA cells in the downsampled gastrualtion atlas 
io$seqFISH_additional_metadata = file.path(io$main, "projects/03_seqFISH_FINAL/code/figure_generation/outputs/metadata/seqFISH_additional_metadata.Rds")
seqFISH_additional_metadata = readRDS(io$seqFISH_additional_metadata)
seqFISH_celltype_for_seqFISH <- subset(seqFISH_additional_metadata, select = postQC_annotations)

# make a combined data.frame with all the seqFISH cell type labels for all the cells 
seqFISH_celltypes <- rbind(seqFISH_celltype_for_seqFISH, seqFISH_celltype_for_scRNA)
colnames(seqFISH_celltypes) <- "seqFISH_celltypes"

# check the list of cell type labels
table(seqFISH_celltypes$seqFISH_celltypes, useNA = "ifany")


        proximal ExE ectoderm #1         proximal ExE ectoderm #2 
                            1680                              980 
          distal ExE ectoderm #1           distal ExE ectoderm #2 
                            5497                             2058 
                 ExE ectoderm #3             visceral endoderm #1 
                              78                             8526 
            visceral endoderm #2                     ExE endoderm 
                             819                              756 
                        brain #1                         brain #2 
                            7368                             6032 
                        brain #3                         brain #6 
                            6210                             1666 
                        brain #5                         brain #4 
                            1204                             1235 
                     spinal cord                     otic pla

In [4]:
#io$seqFISH_to_scRNA = file.path(io$main, "projects/03_seqFISH_FINAL/code/FINAL_METADATA/outputs/total_meta_updated.Rds")

In [5]:
## Load in the seqFISH CellType Labels
io$seqFISH_labels = file.path(io$main, 
                              "projects/03_seqFISH_FINAL/code/stabmap_label_transfer/output/StabMAP_all_stages_post_QC_reweighted_MNNcorrected_original_counts_without_neigh_LABELS.Rds")

seqFISH_labels = readRDS(io$seqFISH_labels)

seqFISH_stage_transfer <- subset(seqFISH_labels, select = stage_transfer)

## Determine the majority cell type labels in each of the cluster.subcluster

In [6]:
# Function to calculate the majority cell type and its proportion
calculate_majority_cluster <- function(data, celltype_col, cluster_col) {
  data %>%
    group_by(!!sym(cluster_col), !!sym(celltype_col)) %>%
    summarise(count = n(), .groups = 'drop') %>%
    group_by(!!sym(cluster_col)) %>%
    mutate(proportion = count / sum(count)) %>%
    arrange(desc(proportion)) %>%
    filter(proportion == max(proportion)) %>%
    summarise(
      celltype = paste(!!sym(celltype_col), collapse = ", "),
      proportion = unique(proportion),
      .groups = 'drop'
    ) %>%
    rename(cluster = !!sym(cluster_col))
}

# Assuming your data.frame is named meta
# Calculate majority cell type for seqFISH_celltype and a cluster column
cluster_column <- "cluster_subcluster" # change this to your desired cluster column

majority_seqFISH <- calculate_majority_cluster(meta, "seqFISH_celltype", cluster_column)
majority_extended_atlas_celltype <- calculate_majority_cluster(meta, "extended_atlas_celltype", cluster_column)
majority_celltype_PijuanSala2019_celltype <- calculate_majority_cluster(meta, "celltype_PijuanSala2019", cluster_column)

# update the column names
colnames(majority_seqFISH) <- c("cluster_subcluster", "majority_seqFISH_celltype", "majority_seqFISH_celltype_proportion")
colnames(majority_extended_atlas_celltype) <- c("cluster_subcluster", "majority_extended_atlas_celltype", "majority_extended_atlas_celltype_proportion")
colnames(majority_celltype_PijuanSala2019_celltype) <- c("cluster_subcluster", "majority_celltype_PijuanSala2019_celltype", "majority_celltype_PijuanSala2019_celltype_proportion")

# make data.frames
majority_seqFISH_df <- as.data.frame(majority_seqFISH)
majority_extended_atlas_celltype_df <- as.data.frame(majority_extended_atlas_celltype)
majority_celltype_PijuanSala2019_celltype_df <- as.data.frame(majority_celltype_PijuanSala2019_celltype)

# generate a new updated meta.data table that includes the majority cell types from the cluster_subclusters
meta_updated <- left_join(meta, majority_seqFISH_df, 
           by = "cluster_subcluster")

meta_updated <- left_join(meta_updated, majority_extended_atlas_celltype_df, 
           by = "cluster_subcluster")

meta_updated <- left_join(meta_updated, majority_celltype_PijuanSala2019_celltype_df, 
           by = "cluster_subcluster")

# set the rownames of the new metadata table to the Cell ID's
rownames(meta_updated) <- meta_updated$cell_ID

## Add the seqFISH cell stage label transfer information to the metadata table

In [7]:
## add the seqFISH stage label transfers to the metadata table
seqFISH_stage_transfer$cell_ID <- rownames(seqFISH_stage_transfer)

meta_updated_seqFISH <- subset(meta_updated, dataset %in% "seqFISH")
meta_updated_scRNA <- subset(meta_updated, dataset %in% "scRNA")

meta_updated_scRNA_stage <- subset(meta_updated_scRNA, select = stage)
colnames(meta_updated_scRNA_stage) <- "stage_transfer"
meta_updated_scRNA_stage$cell_ID <- rownames(meta_updated_scRNA_stage)

stage_transfer <- rbind(meta_updated_scRNA_stage, seqFISH_stage_transfer)

meta_updated <- merge(meta_updated, stage_transfer, by = "cell_ID")

## Perform another round of filtering - give cells from seqFISH with non-matching stage labels the label poor stage mapping

In [8]:
meta_updated_seqFISH <- subset(meta_updated, dataset %in% "seqFISH")
meta_updated_scRNA <- subset(meta_updated, dataset %in% "scRNA")

# Define the stages to include for each condition
stage_E6_5_include <- c("E6.5", "E6.75", "E7.0", "E7.25", "E7.5", "E7.75")
stage_E7_5_include <- c("E6.5", "E6.75", "E7.0", "E7.25","E7.5", "E7.75", "E8.0", "E8.25")
stage_E8_5_include <- c("E8.0", "E8.25", "E8.5", "E8.75", "E9.0", "E9.25", "E9.5")

# Create the poor_stage_alignment column
meta_updated_seqFISH <- meta_updated_seqFISH %>%
  mutate(poor_stage_alignment = case_when(
    stage == "E6.5" & !stage_transfer %in% stage_E6_5_include ~ TRUE,
    stage == "E7.5" & !stage_transfer %in% stage_E7_5_include ~ TRUE,
    stage == "E8.5" & !stage_transfer %in% stage_E8_5_include ~ TRUE,
    TRUE ~ FALSE
  ))

# add the new column with poor stage alignment to the total meta data df

rownames(meta_updated_seqFISH) <- meta_updated_seqFISH$cell_ID
poor_stage_alignment_seqFISH <- subset(meta_updated_seqFISH, select = c(poor_stage_alignment, cell_ID))
row.names(meta_updated) <- meta_updated$cell_ID

meta_updated_scRNA$poor_stage_alignment <- "FALSE"
meta_updated_scRNA <- subset(meta_updated_scRNA, select = c(poor_stage_alignment, cell_ID))
poor_stage_alignment <- rbind(meta_updated_scRNA, poor_stage_alignment_seqFISH)

meta_updated <- merge(meta_updated, poor_stage_alignment, by = "cell_ID")
rownames(meta_updated) <- meta_updated$cell_ID

## Perform quality control and give clusters that only have high levels of seqFISH cells or high levels of scRNA cells ---> poor joint clusters label

In [9]:
# Step 1: Summarize the data to count the number of cells in each cluster_subcluster and dataset
summary_data <- meta_updated %>%
  group_by(cluster_subcluster, dataset) %>%
  summarize(count = n(), .groups = 'drop')

# Step 2: Calculate the proportions of cells from each dataset within each cluster_subcluster
proportion_data <- summary_data %>%
  group_by(cluster_subcluster) %>%
  mutate(proportion = count / sum(count)) %>%
  ungroup()

proportion_data_df <- as.data.frame(proportion_data)
proportion_data_df_seqFISH <- subset(proportion_data_df, dataset %in% "seqFISH")

# Add proportion of seqFISH cells as a new metadata column
meta_updated_2 <- meta_updated %>%
  left_join(proportion_data_df_seqFISH %>% 
              select(cluster_subcluster, proportion) %>% 
              rename(proportion_seqFISH_cells_per_subcluster = proportion), 
            by = "cluster_subcluster")

# Identify clusters that pass QC
joint_clusters_pass_QC <- meta_updated_2 %>%
  filter(proportion_seqFISH_cells_per_subcluster > 0.02 & proportion_seqFISH_cells_per_subcluster < 0.98) %>%
  select(cell_ID)  # Only keep cell_IDs for filtering

# Apply QC filters and create new columns
meta_updated_final <- meta_updated_2 %>%
  mutate(
    joint_clusters_pass_QC = cell_ID %in% joint_clusters_pass_QC$cell_ID,
    poor_stage_alignment = as.logical(poor_stage_alignment),  # Ensure logical type
    stage_alignment_pass_QC = !poor_stage_alignment,
    passed_QC = joint_clusters_pass_QC & stage_alignment_pass_QC,
    refined_annotation = majority_extended_atlas_celltype  # Copy column directly
  )

# Refine the cell type labels for ExE Ectoderm and LPM

In [10]:
meta_updated_6 <- meta_updated_final

# update the ExE ectoderm cell type annotation
meta_updated_6 <- meta_updated_6 %>%
  mutate(refined_annotation = if_else(refined_annotation == "ExE ectoderm", majority_seqFISH_celltype, refined_annotation))

# update the LPM cell type annotation
meta_updated_6 <- meta_updated_6 %>%
  mutate(refined_annotation = if_else(refined_annotation == "Lateral plate mesoderm", majority_seqFISH_celltype, refined_annotation))

rownames(meta_updated_6) <- meta_updated_6$cell_ID

## Perform another round of refining annotations

In [11]:
# load in the final refined_annotations
FINAL_REFINED_ANNOTATIONS <- meta_updated_6

# make refined annotations a character vector
FINAL_REFINED_ANNOTATIONS$refined_annotation <- as.character(FINAL_REFINED_ANNOTATIONS$refined_annotation)

# make seqFISH_celltype a character vector
FINAL_REFINED_ANNOTATIONS$seqFISH_celltype <- as.character(FINAL_REFINED_ANNOTATIONS$seqFISH_celltype)

In [12]:
table(meta_updated_6$refined_annotation)


                                     Allantois 
                                          2720 
                         Allantois endothelium 
                                          1944 
                             Amniotic ectoderm 
                                          1187 
                     Anterior Primitive Streak 
                                          1633 
         Anterior cardiopharyngeal progenitors 
                                           342 
                      Anterior somitic tissues 
                                          1032 
                             Blood progenitors 
                                          2008 
                   Branchial arch neural crest 
                                          2133 
                          Cardiomyocytes FHF 1 
                                          2792 
                          Cardiomyocytes FHF 2 
                                          1094 
                          Cardiomyocyte

In [13]:
df <- FINAL_REFINED_ANNOTATIONS

# Assuming your data frame is called df
df <- df %>%
  mutate(refined_annotation = case_when(
    refined_annotation %in% c("proximal ExE ectoderm #1", "proximal ExE ectoderm #2") ~ "ExE ectoderm distal",
    refined_annotation %in% c("distal ExE ectoderm #1", "distal ExE ectoderm #2") ~ "ExE ectoderm proximal",
    refined_annotation %in% c("mesenchyme") ~ "ExE mesoderm and Anterior LPM",
    refined_annotation %in% c("posterior somatic LP mesoderm #2", "posterior somatic LP mesoderm #1") ~ "ExE mesoderm and Posterior LPM",
    refined_annotation %in% c("lateral plate mesoderm") ~ "ExE mesoderm and Anterior LPM",
    refined_annotation %in% c("Cardiopharyngeal progenitors SHF") ~ "Anterior LPM",
    refined_annotation %in% c("Endocardium") ~ "Endocardium #1",  
    TRUE ~ refined_annotation  # Keep the remaining names unchanged
  ))

# Assuming FINAL_REFINED_ANNOTATIONS is your data frame
df <- df %>%
  mutate(refined_annotation = case_when(
    refined_annotation == "Embryo proper endothelium" & cluster == "19" ~ "Endocardium #2",
    refined_annotation == "Embryo proper endothelium" & cluster == "36" ~ "Embryo proper endothelium #1",
    refined_annotation == "Embryo proper endothelium" & cluster == "49" ~ "Embryo proper endothelium #2",
    refined_annotation == "Embryo proper endothelium" & cluster == "51" ~ "Allantois endothelium",
    TRUE ~ refined_annotation  # Keep the remaining names unchanged
  ))

# Assuming FINAL_REFINED_ANNOTATIONS is your data frame
df <- df %>%
  mutate(refined_annotation = case_when(
    refined_annotation == "Primitive Streak" & cluster_subcluster %in% c("44.4", "2.10", "2.8", "63.0", "2.3", "2.15", "2.12") ~ "Rostral ectoderm",
    refined_annotation == "Epiblast" & cluster_subcluster %in% c("61.5") ~ "ExE endoderm",
    refined_annotation == "Haematoendothelial progenitors" & cluster_subcluster %in% c("44.5") ~ "ExE mesoderm",
    refined_annotation == "Parietal endoderm" ~ "Visceral endoderm",
    TRUE ~ refined_annotation  # Keep the remaining names unchanged
  ))

# Assuming FINAL_REFINED_ANNOTATIONS is your data frame
df <- df %>%
  mutate(seqFISH_celltype = case_when(
    seqFISH_celltype == "ExE ectoderm #3" ~ "haematoendothelial #2",
    seqFISH_celltype == "haematoendothelial" ~ "haematoendothelial #1",
    seqFISH_celltype == "definitive endoderm" ~ "anterior primitive streak",
    seqFISH_celltype == "visceral endoderm #1" ~ "ExE endoderm",
    seqFISH_celltype == "ExE endoderm" ~ "visceral endoderm #1",
    seqFISH_celltype == "proximal ExE ectoderm #1" ~ "distal ExE ectoderm #1",
    seqFISH_celltype == "proximal ExE ectoderm #2" ~ "distal ExE ectoderm #2",
    seqFISH_celltype == "distal ExE ectoderm #1" ~ "proximal ExE ectoderm #1",
    seqFISH_celltype == "distal ExE ectoderm #2" ~ "proximal ExE ectoderm #2",
    TRUE ~ seqFISH_celltype  # Keep the remaining names unchanged
  ))

## Additional (Post Revision - Updated Annotations)

In [14]:
## Cell type annotations to further update 

# Update some ectoderm labels
#Dorsal midbrain neurons = cluster 32, 21 and 46 --> cluster 32 is not in the brain
#Hind brain floor plate = cluster 32 and 55 (32 is along AP, 55 is in brain)
# Neural tube = cluster 5 and 55 (location is in the brain region, which is confusing) 
# Ventral hind brain progenitors = cluster 5 (brain) and cluster 32 (along AP axis)


# Limb mesoderm = cluster 15, 16, 20 and 62, likely contains some LPM 
# Nascent mesoderm = clusters 27, 29 and 44 

# pharyngeal endoderm = cluster 41 (ventral) and cluster 47 (dorsal)

# Update somitic tissue labels
# Endotome is mostly anterior and is made up of cluster 3 (mainly 3) and 26 
# posterior somitic tissues = clsuter 23 (differentiation front) and 4
# Sclerotome anterior cluster = 3 and more posterior = 4 and 49
# Dermomyotome is mostly cluster 4
# Dorsal spinal cord progenitors, Neural tube --> just dorsal spinal cord progenitors
# Non-neural ectoderm, Surfact ectoderm --> Surface ectoderm



# Assuming FINAL_REFINED_ANNOTATIONS is your data frame
df <- df %>%
  mutate(refined_annotation = case_when(
#    refined_annotation == "Dorsal midbrain neurons" & cluster == "32" ~ "Ventral neural tube",
#    refined_annotation == "Hind brain floor plate" & cluster == "32" ~ "Ventral neural tube",
#    refined_annotation == "Ventral hindbrain progenitors" & cluster == "32" ~ "Ventral neural tube",
    refined_annotation == "Dorsal spinal cord progenitors, Neural tube" ~ "Dorsal spinal cord progenitors",
    refined_annotation == "Non-neural ectoderm, Surface ectoderm" ~ "Surface ectoderm", 
    TRUE ~ refined_annotation  # Keep the remaining names unchanged
  ))

# Assuming FINAL_REFINED_ANNOTATIONS is your data frame
#df <- df %>%
#  mutate(refined_annotation = case_when(
#    refined_annotation == "Sclerotome" & cluster == "3" ~ "Cranial mesoderm and anterior somitic tissue",
#    refined_annotation == "Endotome" ~ "Cranial mesoderm and anterior somitic tissue",
#    TRUE ~ refined_annotation  # Keep the remaining names unchanged
#  ))

# Assuming FINAL_REFINED_ANNOTATIONS is your data frame
df <- df %>%
  mutate(refined_annotation = case_when(
    refined_annotation == "Anterior Primitive Streak" & cluster == "47" ~ "Notochord",
    TRUE ~ refined_annotation  # Keep the remaining names unchanged
  ))

## Final Steps and Save

In [15]:
## update the embryo names 
FINAL_REFINED_ANNOTATIONS <- df

# add a column with the embryo ID that is updated
FINAL_REFINED_ANNOTATIONS$embryo <- FINAL_REFINED_ANNOTATIONS$orig.ident

FINAL_REFINED_ANNOTATIONS$embryo <- recode(FINAL_REFINED_ANNOTATIONS$embryo, "embryo1" = "embryo_5",
                        "embryo2" = "embryo_6",
                        "embryo3" = "embryo_7",
                        "embryo6" = "embryo_1",
                        "embryo7" = "embryo_2",
                        "embryo4" = "embryo_3",
                        "embryo5" = "embryo_4",
                            "cell" = "scRNA",
                              "ext" = "scRNA")

In [16]:
table(FINAL_REFINED_ANNOTATIONS$stage_alignment_pass_QC, FINAL_REFINED_ANNOTATIONS$embryo, useNA = "ifany")

       
         scRNA embryo_5 embryo_6 embryo_7 embryo_3 embryo_4 embryo_1 embryo_2
  FALSE      0     2808     2000     2144      189      107      398      368
  TRUE  125559    15487    12420    19092     2725      998     1985     2467

In [17]:
table(FINAL_REFINED_ANNOTATIONS$dataset, FINAL_REFINED_ANNOTATIONS$stage_alignment_pass_QC, useNA = "ifany")

         
           FALSE   TRUE
  scRNA        0 125559
  seqFISH   8014  55174

## Save the FINAL REFINED ANNOTATIONS

In [18]:
#saveRDS(FINAL_REFINED_ANNOTATIONS, "projects/03_seqFISH_FINAL/code/CELL_GENOMICS_REVISIONS/outputs/1B_FINAL_REFINED_ANNOTATIONS_REVISIONS_POST_REVISION.Rds")

## Print out Package Versions

In [19]:
sessionInfo()

R version 4.2.1 (2022-06-23)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.5 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas-pthread/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/liblapack.so.3

locale:
 [1] LC_CTYPE=C.UTF-8    LC_NUMERIC=C        LC_TIME=C          
 [4] LC_COLLATE=C        LC_MONETARY=C       LC_MESSAGES=C      
 [7] LC_PAPER=C          LC_NAME=C           LC_ADDRESS=C       
[10] LC_TELEPHONE=C      LC_MEASUREMENT=C    LC_IDENTIFICATION=C

attached base packages:
[1] grid      stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] png_0.1-8                   Cairo_1.6-0                
 [3] Matrix_1.5-1                viridis_0.6.2              
 [5] viridisLite_0.4.1           cowplot_1.1.1              
 [7] ggpubr_0.4.0                batchelor_1.12.3           
 [9] pheatmap_1.0.12             plotly_4.10.0              
[11] sp_1.5-0      